### Easyun测试环境参数信息
'''
AMI ID(x86): 
    ami-04ad2567c9e3d7893
        AMI name: amzn2-ami-hvm-2.0.20211103.0-x86_64-gp2
    ami-01cc34ab2709337aa
        Platform: Amazon Linux (Inferred)
        AMI name: amzn2-ami-hvm-2.0.20211005.0-x86_64-gp2      
AMI ID(x86): 
    ami-0447addf6c74624eb
        Platform: Amazon Linux (Inferred)
        AMI name: amzn2-ami-hvm-2.0.20211005.0-arm64-gp2
        
Instance type(x86):
    t2.micro
Instance type(arm):     
    t4g.nano
    t4g.micro
    
VPC ID：
    vpc-057f0e3d715c24147 (Easyun-VPC) 
Subnet ID：
     subnet-06bfe659f6ecc2eed (pub-subnet-1, us-east-1a) 
     subnet-02a09fd044f6d8e8d (pub-subnet-2, us-east-1b) 
     subnet-03c3de7f09dfe36d7 (pri-subnet-1, us-east-1a) 
     subnet-0c903785974d075f0 (pri-subnet-2, us-east-1b)      
Security group ID：
    sg-05df5c8e8396d06e9 (easyun-sg-database)
    sg-02f0f5390e1cba746 (easyun-sg-webapp)
    sg-0bb69bb599b303a1e (easyun-sg-dev)
    
Key pair:
    key_easyun_dev
'''

In [11]:
# Easyun环境下新建云服务器
import boto3

# Boto3
# ec2.create_instances(ImageId='<ami-image-id>', MinCount=1, MaxCount=5)

REGION = "us-east-1"
FLAG = 'Easyun'
RESOURCE = boto3.resource('ec2', region_name=REGION)

# 云服务器参数定义
NAME = 'boto3test'
NUM = 1
AMI = 'ami-0447addf6c74624eb'
TYPE = 't4g.nano'
SUBNET = 'subnet-06bfe659f6ecc2eed'
SGs = ['sg-05df5c8e8396d06e9',]
KEY = "key_easyun_dev"
DISK = [
    {
        'DeviceName': '/dev/xvda',
        'Ebs': {            
            'DeleteOnTermination': True,
            'VolumeSize': 16,
            'VolumeType': 'gp2'
            }
    },
    {
        'DeviceName': '/dev/sdf',
        'Ebs': {            
            'DeleteOnTermination': True,
            'VolumeSize': 13,
            'VolumeType': 'gp2'
            } 
    }
]

TAG_SPEC = [
    {
    "ResourceType":"instance",
    "Tags": [
            {"Key": "Flag", "Value": FLAG},
            {"Key": "Name", "Value": NAME}
        ]
    }
    ]


ec2 = boto3.resource('ec2')
server = ec2.create_instances(
    ImageId=AMI,
    InstanceType=TYPE,
    MaxCount=NUM,
    MinCount=NUM,
    BlockDeviceMappings=[
        ,
    ],
    SubnetId=SUBNET,
    SecurityGroupIds=SGs,
    KeyName=KEY,
    TagSpecifications = TAG_SPEC,    
#    Monitoring={'Enabled': False}
)
#print(server)
print(server[0].id, server[0].image.id, server[0].instance_type)
# print(server[0].state["Name"])

i-0a3ee56a27d528525 ami-0447addf6c74624eb t4g.nano


In [14]:
# 查询所有新建的测试服务器
import boto3

REGION = "us-east-1"
TAG = 'boto3test'
RESOURCE = boto3.resource('ec2', region_name=REGION)

servers = RESOURCE.instances.filter(
    Filters=[
#        {'Name': 'instance-state-name', 'Values': ['running']},
        {'Name': 'tag:Name','Values': [TAG]}
    ])
for server in servers:
    # print(server)
    print(server.id, server.instance_type, server.public_ip_address, server.state["Name"])

i-0a3ee56a27d528525 t4g.nano 54.174.204.236 running
